In [ ]:
import os
import numpy as np
import pandas as pd

import seaborn as sea

import tensorflow as tf
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.densenet import preprocess_input
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.regularizers import l2
from keras.utils.vis_utils import plot_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import matthews_corrcoef as MCC
from sklearn.metrics import balanced_accuracy_score as BAS
from sklearn.preprocessing import OneHotEncoder
from sklearn.utils import shuffle
import tensorflow_addons as tfa
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from distutils.dir_util import copy_tree, remove_tree
from tensorflow.keras.preprocessing.image import ImageDataGenerator as IDG

In [ ]:
classes = ["damage", "no_damage"]
path = "/kaggle/input/satellite-images-of-hurricane-damage/train_another"

file_path = [os.path.join(path, "damage/-93.55964_30.895018.jpeg"),
             os.path.join(path, "no_damage/-95.061275_29.831535.jpeg")]

In [ ]:
model = DenseNet121(input_shape=(224,224,3), include_top=True)
METRICS = [tf.keras.metrics.CategoricalAccuracy(name='acc'),
           tf.keras.metrics.AUC(name='auc'), 
           tfa.metrics.F1Score(num_classes=2)]
model.compile(optimizer='adam',
              loss=tf.losses.CategoricalCrossentropy(),
              metrics=METRICS)

model.summary()

In [ ]:
vector = model.get_layer("avg_pool").output
feature_extractor = tf.keras.Model(model.input, vector)

In [ ]:
X_list = []
Y_list = []

for f in range(2):    
    folder_path = os.path.join(path, classes[f])
    for file in os.listdir(folder_path):    
        file_path = os.path.join(folder_path, file)
        
        # check file extension, skip file if not jpg
        if not(file.endswith(".jpeg")):
            continue
        
        # load image
        img = image.load_img(file_path, target_size=(224,224))
        # convert image to numpy array
        img_arr = image.img_to_array(img)
        # add 1 more dimension
        img_arr_b = np.expand_dims(img_arr, axis=0)
        # preprocess image
        input_img = preprocess_input(img_arr_b)
        # extract feature
        feature_vec = feature_extractor.predict(input_img)
    
        X_list.append(feature_vec.ravel())
        Y_list.append(f)

In [ ]:
X = np.asarray(X_list, dtype=np.float32)
Y = np.asarray(Y_list, dtype=np.float32)

for s in range(100):
    X, Y = shuffle(X, Y)
    
print("Shape of feature matrix X")
print(X.shape)
print("\nShape of label matrix Y")
print(Y.shape)

class_types, counts = np.unique(Y, return_counts=True)

print("\nClass labels")
print(class_types)
print("\nClass counts")
print(counts)

In [ ]:
train_X, test_X, train_Y, test_Y = train_test_split(X, Y, test_size=0.2,
                                                    stratify=Y,
                                                    random_state=0)

print("Shape of train_X")
print(train_X.shape)
print("\nShape of test_X")
print(test_X.shape)

In [ ]:
from sklearn.naive_bayes import GaussianNB

GNB_lin = GaussianNB()
GNB_lin.fit(train_X, train_Y)
y_pred = GNB_lin.predict(test_X)
print(classification_report(test_Y, y_pred,
                            target_names=classes))
print(confusion_matrix(test_Y, y_pred))

In [ ]:
import numpy as np
def plot_confusion_matrix(cm,
                          target_names,
                          title='Confusion matrix',
                          cmap=None,
                          normalize=True):
    """
    given a sklearn confusion matrix (cm), make a nice plot

    Arguments
    ---------
    cm:           confusion matrix from sklearn.metrics.confusion_matrix

    target_names: given classification classes such as [0, 1, 2]
                  the class names, for example: ['high', 'medium', 'low']

    title:        the text to display at the top of the matrix

    cmap:         the gradient of the values displayed from matplotlib.pyplot.cm
                  see http://matplotlib.org/examples/color/colormaps_reference.html
                  plt.get_cmap('jet') or plt.cm.Blues

    normalize:    If False, plot the raw numbers
                  If True, plot the proportions

    Usage
    -----
    plot_confusion_matrix(cm           = cm,                  # confusion matrix created by
                                                              # sklearn.metrics.confusion_matrix
                          normalize    = True,                # show proportions
                          target_names = y_labels_vals,       # list of names of the classes
                          title        = best_estimator_name) # title of graph

    Citiation
    ---------
    http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html

    """
    
    import matplotlib.pyplot as plt
    import numpy as np
    import itertools

    accuracy = np.trace(cm) / float(np.sum(cm))
    misclass = 1 - accuracy

    if cmap is None:
        cmap = plt.get_cmap('Blues')

    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()

    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=90)
        plt.yticks(tick_marks, target_names)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]


    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.4f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")


    plt.tight_layout()
    plt.ylabel('ACTUAL', fontsize=18)
    font = {'family' : 'serif',
        'weight' : 'bold'}
    plt.xlabel('PREDICTED',fontsize=12)# label\naccuracy={:0.4f}; misclass={:0.4f}'.format(accuracy, misclass))
    font = {'family' : 'serif',
        'weight' : 'bold'}
    plt.show()

In [ ]:
font = {'family' : 'serif',
        'weight' : 'bold',
        'size'   : 18}
import matplotlib.pyplot as plt
plt.rc('font', **font)
#plt.rcParams({'font.family':'serif', 'font.serif':'Times New Roman'})
plt.rcParams["font.family"] = "serif"
plt.rcParams["font.size"] = 12

In [ ]:
plot_confusion_matrix(cm           = np.array([[855,  145],
                                               [151, 849]]),

                      normalize    = False,
                      target_names = ["Damage", "NonDamage"],
                      title        = "DenseNet121") 



In [ ]:
n_encoder = OneHotEncoder(sparse=False)

# fit encoder to train_Y
n_encoder.fit(train_Y.reshape(-1,1))
# transform train_Y
e_train_Y = n_encoder.transform(train_Y.reshape(-1,1))
# transform test_Y
e_test_Y = n_encoder.transform(test_Y.reshape(-1,1))

In [ ]:
def create_model():
    model = Sequential()
    model.add(Dense(1000, input_dim=1024, activation="relu"))
    model.add(Dropout(0.3))
    model.add(Dense(2, kernel_regularizer=l2(0.1), activation="linear"))
    model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.0001),
                 loss="categorical_hinge",metrics=METRICS)
    return model

epoch = 1000
model = create_model()
history = model.fit(train_X, e_train_Y,
                    validation_split = 0.15,
                    epochs=epoch, batch_size=64, verbose=1)

In [ ]:
#Plotting the trend of the metrics during training

fig, ax = plt.subplots(1, 3, figsize = (20, 10))
ax = ax.ravel()

for i, metric in enumerate(["acc", "auc", "loss"]):
    ax[i].plot(history.history[metric])
    ax[i].plot(history.history["val_" + metric])
    ax[i].set_title("Model {}".format(metric))
    ax[i].set_xlabel("Epochs")
    ax[i].set_ylabel(metric)
    ax[i].legend(["train", "val"])
    
font = {'family' : 'serif',
        'weight' : 'bold',
        'size'   : 12}
import matplotlib
matplotlib.rc('font', **font)
#plt.rcParams({'font.family':'serif', 'font.serif':'Times New Roman'})
plt.rcParams["font.family"] = "serif"

In [ ]:
e = np.linspace(1, epoch, epoch)
fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(8, 6))
sea.lineplot(x = e, y = history.history["loss"],
             ax=axes, label="train");
sea.lineplot(x = e, y = history.history["val_loss"],
             ax=axes, label="val");
axes.set_ylabel("Categorical Hinge Loss")
axes.set_xlabel("epoch");

In [ ]:
y_pred = np.argmax(model.predict(test_X), axis=-1);
print(classification_report(test_Y, y_pred,
                            target_names=classes))
print(confusion_matrix(test_Y, y_pred))

In [ ]:
fig, ax = plt.subplots(2,1)
ax[0].plot(history.history['loss'], color='b', label="Training loss")
ax[0].plot(history.history['val_loss'], color='r', label="Testing loss",axes =ax[0])
legend = ax[0].legend(loc='best', shadow=True)

plt.show()

In [ ]:
import numpy as np
def plot_confusion_matrix(cm,
                          target_names,
                          title='Confusion matrix',
                          cmap=None,
                          normalize=True):
    """
    given a sklearn confusion matrix (cm), make a nice plot

    Arguments
    ---------
    cm:           confusion matrix from sklearn.metrics.confusion_matrix

    target_names: given classification classes such as [0, 1, 2]
                  the class names, for example: ['high', 'medium', 'low']

    title:        the text to display at the top of the matrix

    cmap:         the gradient of the values displayed from matplotlib.pyplot.cm
                  see http://matplotlib.org/examples/color/colormaps_reference.html
                  plt.get_cmap('jet') or plt.cm.Blues

    normalize:    If False, plot the raw numbers
                  If True, plot the proportions

    Usage
    -----
    plot_confusion_matrix(cm           = cm,                  # confusion matrix created by
                                                              # sklearn.metrics.confusion_matrix
                          normalize    = True,                # show proportions
                          target_names = y_labels_vals,       # list of names of the classes
                          title        = best_estimator_name) # title of graph

    Citiation
    ---------
    http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html

    """
    
    import matplotlib.pyplot as plt
    import numpy as np
    import itertools

    accuracy = np.trace(cm) / float(np.sum(cm))
    misclass = 1 - accuracy

    if cmap is None:
        cmap = plt.get_cmap('Blues')

    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()

    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=90)
        plt.yticks(tick_marks, target_names)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]


    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.4f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")


    plt.tight_layout()
    plt.ylabel('ACTUAL', fontsize=18)
    font = {'family' : 'serif',
        'weight' : 'bold'}
    plt.xlabel('PREDICTED',fontsize=12)# label\naccuracy={:0.4f}; misclass={:0.4f}'.format(accuracy, misclass))
    font = {'family' : 'serif',
        'weight' : 'bold'}
    plt.show()

In [ ]:
font = {'family' : 'serif',
        'weight' : 'bold',
        'size'   : 18}
import matplotlib.pyplot as plt
plt.rc('font', **font)
#plt.rcParams({'font.family':'serif', 'font.serif':'Times New Roman'})
plt.rcParams["font.family"] = "serif"
plt.rcParams["font.size"] = 12

In [ ]:
plot_confusion_matrix(cm           = np.array([[975,  25],
                                               [52, 948]]),

                      normalize    = False,
                      target_names = ["Damage", "NonDamage"],
                      title        = "DenseNet121_GNB")